# Join all the scored files together

In [1]:
import glob
import os
import pandas as pd
from tqdm import tnrange, tqdm_notebook

In [2]:
input_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/sailor_outputs_individual_barcodes_merged_scores_no_APO_filter/'
output_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/outputs'

In [3]:
all_scored = sorted(glob.glob(os.path.join(input_dir, 'possorted_genome_bam_MD*.exons.merged.txt')))

print(len(all_scored))
all_scored[:3]

16946


['/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/sailor_outputs_individual_barcodes_merged_scores_no_APO_filter/possorted_genome_bam_MD-AAACCCAAGAGCCCAA-1.fx.bed.annotated.exons.merged.txt',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/sailor_outputs_individual_barcodes_merged_scores_no_APO_filter/possorted_genome_bam_MD-AAACCCAAGAGCTTTC-1.fx.bed.annotated.exons.merged.txt',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/sailor_outputs_individual_barcodes_merged_scores_no_APO_filter/possorted_genome_bam_MD-AAACCCAAGATAGTGT-1.fx.bed.annotated.exons.merged.txt']

In [4]:
def format_name(fn):
    return os.path.basename(fn).replace('.fx.bed.annotated.exons.merged.txt','')

# I'm doing these merges in batches since merging over 20k cells takes too long
- better to merge 500 at a time and then merge each of the 34 dataframes together.

In [5]:
def chunker(seq, size):
    """
    Chunks a long list into groups of (size).
    """
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

all_merged = []
groupsize = 500 # 50 sailor runs per job.
progress = tnrange(len(all_scored))
for group in chunker(all_scored, groupsize):
    merged = pd.DataFrame()
    for scored in group:
        df = pd.read_csv(scored, index_col=0, sep='\t')[['edited_over_edited_c']]
        df.columns = [format_name(scored)]
        merged = pd.merge(merged, df, how='outer', left_index=True, right_index=True)
        progress.update(1)
    all_merged.append(merged)

In [6]:
len(all_merged)

34

In [8]:
print(all_all_merged.shape)
all_all_merged.fillna(0, inplace=True)
all_all_merged.to_csv(os.path.join(output_dir, 'possorted_genome_bam_MD.exons.merged.edited_over_edited_c.tsv'), sep='\t', header=True, index=True)

(2087, 16946)
